|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 2:</h2>|<h1>Large language models<h1>|
|<h2>Section:</h2>|<h1>Fine-tune pretrained models<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeGen for code completion<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Import and inspect the CodeGen model

In [ ]:
# Model source:
# https://huggingface.co/Salesforce/codegen-350M-mono
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codegen-350M-mono')

model = AutoModelForCausalLM.from_pretrained('Salesforce/codegen-350M-mono').to(device)

In [ ]:
model

# Check out the tokenizer

In [ ]:
# A bit about their tokenizer
print(f'Tokenizer has {tokenizer.vocab_size:,} tokens.\nA few random tokens:\n')

for i in range(30):
  # generate a random token
  randtok = torch.randint(tokenizer.vocab_size,(1,))
  print(f'Token {randtok[0]:5} is "{tokenizer.decode(randtok)}"')

# Generate a response to some code input

In [ ]:
text = 'for i in range(10):'
input_ids = tokenizer(text, return_tensors='pt').input_ids.to(device)

generated_ids = model.generate(input_ids, max_length=222, temperature=1.4, do_sample=True, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# Import git-calc files and tokenize

In [ ]:
import os
import subprocess
import nbformat

# download the repo
repo_url = 'https://github.com/mikexcohen/Calculus_book.git'
repo_dir = 'Calculus_book'
subprocess.run(['git','clone',repo_url,repo_dir])

# initialize a list for all tokens
all_tokens = []


## find all .ipynb files
for root, dirs, files in os.walk(repo_dir):
  for file in files:
    if file.endswith('.ipynb'):

      # load and process notebooks
      # print(f'Processing notebook: {os.path.join(root, file)}')
      with open(os.path.join(root, file),'r',encoding='utf-8') as f:

        # parse the notebook structure
        nb_data = nbformat.read(f,as_version=4)

        # gather all code cells into a single string
        code_cells = []
        for cell in nb_data['cells']:
          if cell['cell_type'] == 'code': # this cell contains code
            code_cells.append(cell['source']) # the actual code in this cell

        # join code cells
        all_code = '\n'.join(code_cells)

        # tokenize and add to the list of all tokens
        all_tokens += tokenizer(all_code)['input_ids']

# check the final dataset size
print(f'\n\nTraining data contains {len(all_tokens):,} tokens, of which {len(set(all_tokens)):,} are unique.')